In [2]:
from gallerysearch.index import TagIndex

"""Experimenting with trying to get to semantic search by embedding tags, but doesn't work very well"""

i = TagIndex('/home/josh/Pictures/gallery-dl')


Loading tags:   0%|          | 1/21342 [00:51<303:41:16, 51.23s/it]

KeyboardInterrupt: 

In [56]:
import faiss
from sentence_transformers import SentenceTransformer

tags = [x.replace('_', ' ') for x in i.pairs_by_tag.keys()]
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode tags into embeddings
embeddings = model.encode(tags, convert_to_numpy=True, show_progress_bar=True)

# Create a FAISS index
d = embeddings.shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(d)  # L2 (Euclidean) distance index
index.add(embeddings)  # Add embeddings to the index

Batches: 100%|██████████| 268/268 [00:19<00:00, 13.75it/s]


In [61]:
from IPython.display import display
from PIL import Image
import math

# Function to search the index
def search_faiss(query, k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_embedding, k)
    return [(tags[idx], distances[0][i]) for i, idx in enumerate(indices[0])]

# Example Query
query = "beach and ocean"
results = search_faiss(query, k=len(tags))

# Print results
tag_scores = {x: float(y) for x,y in results}

def image_score(img_tags: list[str]):
    img_tags = [x.replace('_', ' ') for x in img_tags]
    scores = [tag_scores.get(tag, 0) for tag in img_tags]

    # return average of top 5 tags
    #return sum(sorted(scores, reverse=True)[:5]) / 5

    return sum(scores) / len(scores)

scored_images = [
    (pair, image_score(img_tags))
    for pair, img_tags in i.tags_by_pair.items()
]
scored_images.sort(key=lambda x: x[1], reverse=True)

# List of image paths
image_paths = [x[0].img_file for x in scored_images[:5]]

# Open and display images
new_size = (300, 300)

# Open, resize, and display images
images = [Image.open(path).resize(new_size).convert("RGB") for path in image_paths]
display(*images)
print(image_paths)

KeyboardInterrupt: 